In [1]:
import pandas as pd
import torch
print(torch.__version__)

2.1.2+cu121


In [2]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        print(f'Using device: {torch.cuda.get_device_name(0)}')
        return torch.device('cuda')
    elif torch.backends.mps.is_available():
        print('Using device: Apple ARM GPU')
        return torch.device('mps')
    else:
        print('Using device: CPU')
        return torch.device('cpu')
    
# Set the device
device = get_default_device()
print(f'Using device: {device}')

Using device: NVIDIA GeForce RTX 3060
Using device: cuda


In [3]:
processed_df = pd.read_csv('../data/csv/processed.csv')
processed_df

,wind_speed_TSN,wind_dir_TSN,wind_speed_KADUM,wind_dir_KADUM,wind_speed_PATNA,wind_dir_PATNA,wind_speed_SADAS,wind_dir_SADAS,wind_speed_DADEN,wind_dir_DADEN,...,Route,Ac_id,Ac_code,Ac_type,Ac_kts,Time_step,Phase,Ac_Lat,Ac_Lon,Ac_feet
0,0.630435,0.363611,0.200000,0.361868,0.229167,0.561418,0.014493,0.993860,0.150000,0.884247,...,6,R6-VJ150-21,VJ150,1,170.0,3,1,-0.931666,-0.090884,0.012828
1,0.630435,0.363611,0.200000,0.362146,0.229167,0.562535,0.014493,0.994628,0.150000,0.882877,...,6,R6-VJ150-21,VJ150,1,179.0,23,1,-0.932750,-0.101588,0.032376
2,0.630435,0.363611,0.200000,0.362146,0.229167,0.562535,0.014493,0.994628,0.150000,0.882877,...,6,R6-VJ150-21,VJ150,1,205.0,43,1,-0.933595,-0.113597,0.041539
3,0.630435,0.363611,0.200000,0.362424,0.229167,0.563652,0.000000,0.996163,0.150000,0.882192,...,6,R6-VJ150-21,VJ150,1,231.0,59,1,-0.932970,-0.124232,0.047037
4,0.630435,0.363611,0.200000,0.362424,0.229167,0.563652,0.000000,0.996163,0.150000,0.882192,...,6,R6-VJ150-21,VJ150,1,255.0,75,1,-0.930379,-0.133221,0.058033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61383,0.347826,0.381111,0.327273,0.118121,0.531250,0.101619,0.391304,0.629317,0.533333,0.580822,...,6,R6-VN258-04,VN258,2,157.0,5344,1,0.986587,-0.746252,0.040929
61384,0.347826,0.381111,0.327273,0.118121,0.531250,0.101619,0.391304,0.629317,0.533333,0.580822,...,6,R6-VN258-04,VN258,2,152.0,5362,1,0.985870,-0.737057,0.034820
61385,0.347826,0.381111,0.327273,0.117009,0.531250,0.101619,0.391304,0.629317,0.533333,0.580822,...,6,R6-VN258-04,VN258,2,146.0,5378,1,0.985282,-0.729440,0.029933
61386,0.347826,0.381111,0.327273,0.117009,0.531250,0.101619,0.391304,0.629317,0.533333,0.580822,...,6,R6-VN258-04,VN258,2,131.0,5412,1,0.984180,-0.715031,0.020770
